Le but de ce notebook est de comparer le temps d'exécution de nos algorithmes en fonction de la taille de l'entrée lorsqu'on prend des matrices aléatoires.

# Importation des modules nécessaires

In [ ]:
import pickle
import time
import clustering_method as cls
import gradient_descent_script as gd 
import alternating_minimization as alt
from utilities import generate_low_rank, generate_entries
import convex_optimization as cvx
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Variables

In [ ]:
T=range(50,601,50) #tailles considérées
complete_matrices = [generate_low_rank(n,n,int(n**(1/5))) for n in T] #Génération des matrices complètes de rang int(n^(1/5)) pour chaque taille n
n=len(complete_matrices)
entries = [generate_entries(n,n,int(5*n*np.log(n))) for n in T] #Génération des entrées à révéler int(5nlog(n)) pour chaque taille n

# Traçage des courbes

## Clustering

In [ ]:
duration_clustering = []
for i in tqdm(range(n)):
    debut_clustering = time.time() 
    incomplete_m=complete_matrices[i]
    L=tuple(zip(*entries[i]))
    incomplete_m[L]=np.zeros(len(entries[i]))
    complete_m = cls.complete_matrix(incomplete_m)
    fin_clustering = time.time()
    duration_clustering.append(fin_clustering-debut_clustering)

In [ ]:
with open("predicted_time_clustering", "wb") as f:
    pickle.dump(duration_clustering, f)

## Optimisation convexe

In [ ]:
duration_cvx = []
for i in tqdm(range(n)):
    Y=complete_matrices[i]
    list_of_entries=entries[i]
    debut_cvx = time.time() 
    complete_m = cvx.test_convex(Y,list_of_entries)
    fin_cvx = time.time()
    duration_cvx.append(fin_cvx-debut_cvx)

In [ ]:
with open("predicted_time_cvx", "wb") as f:
    pickle.dump(duration_cvx, f)

## Descente du gradient

In [ ]:
duration_gd = []
for i in tqdm(range(n)):
    Y=complete_matrices[i]
    list_of_entries=entries[i]
    debut_gd = time.time() 
    complete_m = gd.generate_prediction_noprint(Y,list_of_entries,rank=int(Y.shape[0]**(1/5)), learning_rate=0.1)
    fin_gd = time.time()
    duration_gd.append(fin_gd-debut_gd)

In [ ]:
with open("predicted_time_gd", "wb") as f:
    pickle.dump(duration_gd, f)

## Minimisation alternée

In [ ]:
duration_alt = []
for i in tqdm(range(n)):
    Y=complete_matrices[i]
    list_of_entries=entries[i]
    M=gd.generate_M(Y,list_of_entries)
    p,m=Y.shape
    debut_alt = time.time() 
    complete_m = alt.AltMinComplete(list_of_entries,M,p,m,k=int(p**(1/5)))
    fin_alt = time.time()
    duration_alt.append(fin_alt-debut_alt)

In [ ]:
with open("predicted_time_alt", "wb") as f:
    pickle.dump(duration_alt, f)

In [ ]:
plt.plot(T,duration_cvx,label="Optimisation convexe")
plt.plot(T,duration_gd,label="Descente du gradient")
plt.plot(T,duration_alt,label="Minimisation alternée")
plt.plot(T,duration_clustering,label="Clustering")
plt.xlabel('Taille de la matrice')
plt.ylabel("temps (min)")
plt.title("Temps d'exécution en fonction de la taille")
plt.legend()
plt.show()